In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import itertools
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

from transformers import TFAutoModel, AutoTokenizer, BertConfig, TFBertModel

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X1r.csv', sep=';', encoding='cp1251')
df2 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X2_result.csv', sep=';', encoding='cp1251')
df2['relevance'] = df2['predict']

df = pd.concat([df1[['match_id', 'team1', 'team2', 'broadcast_name', 'match_time', 'content', 'id', 'news_name', 'ctime', 'body', 'after_match', 'relevance']],
                df2[['match_id', 'team1', 'team2', 'broadcast_name', 'match_time', 'content', 'id', 'news_name', 'ctime', 'body', 'after_match', 'relevance']]])
print(df.shape)
df.head()

(1849, 12)


,match_id,team1,team2,broadcast_name,match_time,content,id,news_name,ctime,body,after_match,relevance
0,830226,Валенсия,Динамо Киев,Лига Европы,27.02.2014 20:05,"Добрый вечер, друзья! Сегодня нас ждет ответны...",1.582876e+08,"Мбокани: «Если на Украине ничего не изменится,...",28.02.2014 16:28,Нападающий киевского «Динамо» Дьёмерси Мбокани...,1,0
1,828485,Россия,Сербия,Товарищеские матчи (сборные),15.11.2013 16:30,Сегодня в Объединенных Арабских Эмиратах сборн...,1.543978e+08,Алексей Ионов: «Хотели победить и выкладывалис...,15.11.2013 23:38,Полузащитник сборной России Алексей Ионов поде...,1,1
2,1032440,Арсенал,Ливерпуль,Англия. Премьер-лига,24.08.2015 19:00,"Добрый вечер, дамы и господа! Sports.ru предла...",1.032461e+09,Мартин Шкртел: два тайма игры с «Арсеналом» пр...,25.08.2015 16:03,Защитник «Ливерпуля» Мартин Шкртел прокомменти...,1,1
3,964506,Зенит,АЕЛ,Лига чемпионов,06.08.2014 16:00,Всем привет. Петербургский лигочемпионский пут...,1.022678e+09,"Юрий Лодыгин: «Были готовы к тому, что соперни...",06.08.2014 23:47,Голкипер «Зенита» Юрий Лодыгин поделился мнени...,1,1
4,295991,Спартак,ЦСКА,Россия. Олимп Кубок России,11.05.2011 15:30,Всем привет! Продолжаем серию мировых классико...,9.677123e+07,"Сейду Думбия: «Меня абсолютно не удивляет, что...",12.05.2011 9:37,Ивуарийский нападающий ЦСКА Сейду Думбия после...,1,1


In [ ]:
df['body'] = df['body'].apply(lambda x: str(x))
df['len_news_name'] = df['news_name'].apply(lambda x: len(x))
df['len_body'] = df['body'].apply(lambda x: len(x))
df['len_content'] = df['content'].apply(lambda x: len(x))
df.describe()

,match_id,id,after_match,relevance,len_news_name,len_body,len_content
count,1.849000e+03,1.849000e+03,1849.000000,1849.000000,1849.000000,1849.000000,1849.000000
mean,6.190958e+05,4.171134e+08,0.777177,0.823148,72.061114,846.580855,9489.890211
std,3.424326e+05,4.305191e+08,0.416253,0.381647,20.063213,1030.904032,2573.500569
min,1.893600e+04,7.006881e+06,0.000000,0.000000,20.000000,59.000000,24.000000
25%,2.954310e+05,1.056714e+08,1.000000,1.000000,58.000000,447.000000,7575.000000
50%,6.581190e+05,1.479490e+08,1.000000,1.000000,70.000000,652.000000,9199.000000
75%,9.628700e+05,1.024029e+09,1.000000,1.000000,84.000000,967.000000,11115.000000
max,1.104865e+06,1.040573e+09,1.000000,1.000000,162.000000,17571.000000,21648.000000


In [ ]:
### UTILITY FUNCTIONS FOR TOKENIZATIONS, MASKS AND SEGMENTS CREATION ###
### from: https://www.kaggle.com/akensert/bert-base-tf2-0-now-huggingface-transformer

def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    

def convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=True):
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    if double:
    
        input_ids_1, input_masks_1, input_segments_1 = return_id(
            str1, None, 'longest_first', max_sequence_length)

        input_ids_2, input_masks_2, input_segments_2 = return_id(
            str2, None, 'longest_first', max_sequence_length)

        return [input_ids_1, input_masks_1, input_segments_1,
                input_ids_2, input_masks_2, input_segments_2]
    
    else:
        
        input_ids, input_masks, input_segments = return_id(
            str1, str2, 'longest_first', max_sequence_length)

        return [input_ids, input_masks, input_segments,
                None, None, None]        

def compute_input_arrays(df, columns, tokenizer, max_sequence_length, double=True):
    
    input_ids_1, input_masks_1, input_segments_1 = [], [], []
    input_ids_2, input_masks_2, input_segments_2 = [], [], []
    for _, instance in tqdm(df[columns].iterrows(), total=len(df)):
        str1, str2 = instance[columns[0]], instance[columns[1]]

        ids_1, masks_1, segments_1, ids_2, masks_2, segments_2 = \
        convert_to_transformer_inputs(str1, str2, tokenizer, max_sequence_length, double=double)
        
        input_ids_1.append(ids_1)
        input_masks_1.append(masks_1)
        input_segments_1.append(segments_1)

        input_ids_2.append(ids_2)
        input_masks_2.append(masks_2)
        input_segments_2.append(segments_2)
        
    if double:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32),
                np.asarray(input_ids_2, dtype=np.int32), 
                np.asarray(input_masks_2, dtype=np.int32), 
                np.asarray(input_segments_2, dtype=np.int32)]
    
    else:
        
        return [np.asarray(input_ids_1, dtype=np.int32), 
                np.asarray(input_masks_1, dtype=np.int32), 
                np.asarray(input_segments_1, dtype=np.int32)]

In [ ]:
count_tokens = 300
res = pd.DataFrame(np.repeat(df.values, df['len_body'].apply(lambda x: x // count_tokens + 1).array, axis=0), columns=df.columns).astype(df.dtypes)
res['numb_start'] = res.groupby(['id']).cumcount()+1
res['part_content'] = res.apply(lambda row: str(row['content'])[(row['numb_start'] - 1) * count_tokens : (row['numb_start']) * count_tokens], axis = 1)
res['part_content'] = res['part_content'].apply(lambda x: str(x))
res['news_name'] = res['news_name'].apply(lambda x: str(x))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(res[['part_content', 'body']], res['relevance'].values, 
                                                    random_state=42, test_size = 0.2, shuffle=True, stratify=res['relevance'].values)


 
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(4941, 2) (1236, 2)
(4941,) (1236,)


In [ ]:
### IMPORT TOKENIZER ###
count_tokens = 300

MAX_SEQUENCE_LENGTH_CONTENT = count_tokens
MAX_SEQUENCE_LENGTH_NEWS = count_tokens

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base")

In [ ]:
### CREATE SEQUENCES (id, mask, segments) FOR TRAIN AND TEST ###

input_train = compute_input_arrays(X_train, ['part_content', 'body'], tokenizer, MAX_SEQUENCE_LENGTH_CONTENT)
input_test = compute_input_arrays(X_test, ['part_content','body'], tokenizer, MAX_SEQUENCE_LENGTH_NEWS)

  0%|          | 0/4941 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


  0%|          | 0/1236 [00:00<?, ?it/s]

In [ ]:
def dual_bert():
    
    set_seed(33)
    
    opt = Adam(learning_rate=2e-5)
    
    id1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    id2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    mask1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    mask2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    atn1 = Input((MAX_SEQUENCE_LENGTH_CONTENT,), dtype=tf.int32)
    atn2 = Input((MAX_SEQUENCE_LENGTH_NEWS,), dtype=tf.int32)
    
    config = BertConfig() 
    config.output_hidden_states = False # Set to True to obtain hidden states
    bert_model1 = TFBertModel.from_pretrained('sberbank-ai/ruBert-base', from_pt=True)
    bert_model2 = TFBertModel.from_pretrained('sberbank-ai/ruBert-base', from_pt=True)
    
    embedding1 = bert_model1(id1, attention_mask=mask1, token_type_ids=atn1)[0]
    embedding2 = bert_model2(id2, attention_mask=mask2, token_type_ids=atn2)[0]
    
    x1 = GlobalAveragePooling1D()(embedding1)
    x2 = GlobalAveragePooling1D()(embedding2)
    
    x = Concatenate()([x1, x2])
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(2, activation='softmax')(x)

    model = Model(inputs=[id1, mask1, atn1, id2, mask2, atn2], outputs=out)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt)
    
    return model

In [ ]:
model = dual_bert()
model.load_weights('/content/drive/MyDrive/nir/bert2_weights_300_2_relevance')

model.fit(input_train, y_train, epochs=2, batch_size=4)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

Epoch 1/2
1236/1236 [==============================] - 846s 659ms/step - loss: 0.0849
Epoch 2/2
1236/1236 [==============================] - 818s 661ms/step - loss: 0.0199


In [ ]:
### PREDICT TEST ###

pred_test = np.argmax(model.predict(input_test), axis=1) 

In [ ]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       292
           1       1.00      1.00      1.00       944

    accuracy                           1.00      1236
   macro avg       1.00      1.00      1.00      1236
weighted avg       1.00      1.00      1.00      1236



In [ ]:
# два берта 300, 3 эпохи, 8000 строк
model.save_weights('/content/drive/MyDrive/nir/bert2_weights_300_2_relevance2')

In [ ]:
model = dual_bert()
model.load_weights('/content/drive/MyDrive/nir/bert2_weights_300_2_relevance2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
df_1 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X1.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X2.csv')
df_3 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X3.csv')
df_4 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X4.csv')
df_5 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X5.csv')

df2 = pd.concat([df_1, df_2, df_3, df_4, df_5])
print(df2.shape)

df2['body'] = df2['body'].apply(lambda x: str(x))
df2['len_news_name'] = df2['news_name'].apply(lambda x: len(x))
df2['len_body'] = df2['body'].apply(lambda x: len(x))
df2['len_content'] = df2['content'].apply(lambda x: len(x))
df2.describe()

count_tokens = 300
res = pd.DataFrame(np.repeat(df2.values, df2['len_body'].apply(lambda x: x // count_tokens + 1).array, axis=0), columns=df2.columns).astype(df2.dtypes)
res['numb_start'] = res.groupby(['id']).cumcount()+1
res['part_content'] = res.apply(lambda row: str(row['content'])[(row['numb_start'] - 1) * count_tokens : (row['numb_start']) * count_tokens], axis = 1)
res['part_content'] = res['part_content'].apply(lambda x: str(x))
res['news_name'] = res['news_name'].apply(lambda x: str(x))

(92997, 12)


In [ ]:
print(res.shape)

(306383, 17)


In [ ]:
input_test = compute_input_arrays(res, ['part_content','body'], tokenizer, MAX_SEQUENCE_LENGTH_NEWS)

  0%|          | 0/20500 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
arr_pred_test = []
for i in range(31):
  input_test = compute_input_arrays(res.iloc[i * 10000:(i + 1) * 10000], ['part_content','body'], tokenizer, MAX_SEQUENCE_LENGTH_NEWS)
  arr_pred_test.append(np.argmax(model.predict(input_test), axis=1))

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

...

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/6383 [00:00<?, ?it/s]

In [ ]:
pred_test = np.vstack((arr_pred_test))

ValueError: ignored

In [ ]:
res['predict'] = pred_test
res

In [ ]:
temp = res.groupby(['id', 'match_id'], as_index=False)['predict'].mean()
temp['predict'] = temp['predict'].apply(lambda x: 1 if x >= 0.5 else 0)
temp

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/NIR/data/X3.csv')

In [ ]:
df2 = df2.merge(temp, how='inner', on=['match_id', 'id'])
df2.to_csv("/content/drive/MyDrive/ColabNotebooks/NIR/result.csv")

In [ ]:
df2.to_csv("/content/drive/MyDrive/ColabNotebooks/NIR/result.csv")

In [ ]:
df2['predict'].value_counts()